In [ ]:
import math 
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite
import networkx as nx
from ortools.linear_solver import pywraplp as OR
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import copy
import pickle
from bokeh import palettes
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import (GraphRenderer, Circle, MultiLine, StaticLayoutProvider,
                          HoverTool, TapTool, EdgesAndLinkedNodes, NodesAndLinkedEdges,
                          ColumnDataSource, LabelSet, NodesOnly)
from bipartite_matching import *
output_notebook()

# Taxi Routing Bipartite Matching Lab

**Objectives:**

* Construct the taxi-routing problem as a bipartite graph.
* Use the maximum cardinality matching problem to solve the taxi-routing problem.
* Compare the optimal solution to the taxi-routing problem with the original routing.
    
<font color='red'> **Instructor Comments** </font>

<font color='blue'> **Solutions** </font>

## Part 1: Bipartite Graph Formulation 

Load the taxi trips information as well as NYC street nodes and arcs.

In [ ]:
trips_df = pd.read_csv('data/2013-09-01_trip_data_manhattan.csv').drop(columns='id')
nodes_df = pd.read_csv('data/nyc_nodes_manhattan.csv').drop(columns='Unnamed: 0')
arcs_df = pd.read_csv('data/nyc_links_manhattan.csv').drop(columns='Unnamed: 0')
times_df = pd.read_csv('data/times.csv', index_col =0)
times_df.columns = times_df.columns.astype(float)

Look at an example input. Each trip consists of the unique vehicle id (medallion), location id of the pickup node and drop-off node, pickup time and drop-off time.

In [ ]:
# A list of example trip_ids
ex_trips = [68326, 69501, 70802, 68619, 69802, 70142, 68751, 69558, 70296, 68272]
# Locate the corresponding trip information
trips = trips_df.iloc[ex_trips]
trips

Consider a bipartite graph $G = (D, P, E)$ where $D$ and $P$ are disjoint sets corresponding to the "left" and "right" sides of the graph, and $E$ is the set of edges, each of which has exactly one endpoint in $D$ and one endpoint in $P$.

Let $D = \{(d_1, T^d_{1}),(d_2, T^d_{2}),\ldots,(d_n, T^d_{n})\}$ denote the set of drop-off nodes, $P = \{(p_1, T^p_{1}),(p_2, T^p_{2}),\ldots,(p_n, T^p_{n})\}$ denote the set of pickup nodes, where there are $n$ taxi trips to be covered. We can express the set of taxi trips $N = \{(p_i, T^p_{i}, d_i, T^d_{i}): i\in \{1,\ldots,n\}\}$; that is, the taxi history shows that the $i$th trip picks up a passenger at time $T^p_{i}$ at point $p_i$ and drops off the fare at time $T^d_{i}$ at point $d_i$.

Construct nodes and edges for the bipartite graph.

In [ ]:
# Intialize nodes and edges
DO_nodes = list()
PU_nodes = list()
edges = list()
# Initialize a dict that maps a PU node to a DO node
PUtoDO = dict()

Each node is a tuple of (location_id, time, trip_id, "DO"/"PU")

In [ ]:
for index, row in trips.iterrows():
    s = row['start_node']
    t = row['end_node']
    s_t = row['start_time']
    t_t = s_t + row['trip_time']
    DO_node = (int(t), t_t, index, 'DO')
    PU_node = (int(s), s_t, index, 'PU')
    DO_nodes.append(DO_node)
    PU_nodes.append(PU_node)
    PUtoDO[PU_node] = DO_node

**Q:** Assume that a taxi handles two consecutive trips $i$ and $j$ (in order), where the drop-off time at location $d_i$ is $T^d_i$ and the pickup time at location $p_j$ is $T^p_j$. What is the *elapsed time* between the drop-off time of trip $i$ and the pickup time of trip $j$ (express in terms of $T^d_i$ and $T^p_j$) ?

**A:** <font color='blue'> The *elapsed time* is $T^p_j - T^d_i$.</font>

**Q:** Assume that $time(d_i, p_j)$ is the time needed to travel between location point $d_i$ and location point $p_j$. What relationship must hold between the *elasped time* between the drop-off time of trip $i$ and the pickup time of trip $j$ and $time(d_i, p_j)$ in order for a taxi to cover both trip $i$ and trip $j$ (express in terms of $time(d_i, p_j), T^d_i$ and $T^p_j$)? What does this inequality mean?

**A:** <font color='blue'> $time(d_i, p_j) \leq T^p_j - T^d_i$. It means that a taxi can reach the new pickup at $p_j$ in time after dropping off the fare at $d_i$. </font>

**Q:** In the previous question, what if the taxi can only stay for a maximum of $\delta$ minutes? What is the new inequality and what does it mean?

**A:** <font color='blue'> $T^p_j - T^d_i - \delta \leq time(d_i, p_j) \leq T^p_j - T^d_i$. The new inequality means that a taxi can reach the new pickup at $p_j$ in time after dropping off the fare at $d_i$, and also doesn’t need to wait more than $\delta$ minutes for the new pickup to be ready. </font>

**Q:** Based on the new inequality you just estabilished, can you define the set $E$ mathematically?

**A:** <font color='blue'> $$E = \{\{(d_i, T^d_{i}),(p_j, T^p_{j})\}: T^p_{j} - T^d_{i}-\delta \leq time(d_i, p_j) \leq T^p_{j} - T^d_{i}\}$$ </font>

In [ ]:
# Sort the nodes by time
DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
PU_nodes = sorted(PU_nodes, key = lambda x: x[1])

# Specify the edges
max_waiting_time = 10 # delta

for DO_node in DO_nodes:
    for PU_node in PU_nodes:
        if PU_node[1] > DO_node[1] + max_waiting_time:
            break
        else:
            if PU_node[1] >= DO_node[1]:
                time = times_df.at[(DO_node[0], PU_node[0])]
                if ((PU_node[1] - DO_node[1]) - max_waiting_time  <= time) & (time <= (PU_node[1] - DO_node[1])):
                    edges.append((DO_node, PU_node))

Generate the bipartite graph

In [ ]:
B = nx.Graph()
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(DO_nodes, bipartite=0)
B.add_nodes_from(PU_nodes, bipartite=1)
# Add edges only between nodes of opposite node sets
B.add_edges_from(edges)

**Theorem**

If a maximum cardinality matching is of size $m$, then the minimum number of taxis needed to cover all $n$ trips is $n-m$.

In [ ]:
match = nx.bipartite.maximum_matching(B, DO_nodes)
print('Size of max cardinality matching:', int(len(match)/2)) # divided by two because the output edges are directed

**Q:** What is the minimum number of taxis needed to cover all the trips according to the size of the maximum cardinality matching?

**A:** <font color='blue'> 4 taxis.</font>

We can use the information given by maximum cardinality matching to track down the optimal taxi trajectories. 

In [ ]:
opt_paths = match_to_path(match, trips)

Plot the bipartite graph

In [ ]:
plot_ex_bipartite(B, match, opt_paths, True)

**Q:** Interpret one optimal taxi trajectory that contains three trips based on the graph above using the location ids (remember each node is a tuple of (location_id, time, trip_id, "DO"/"PU")).

**A:** <font color='blue'> 1447->1762->1163->24->79->931 (or 2119->275->1494->1809->20->1323)</font>

Plot the corresponding taxi paths on the map

In [ ]:
G = street_network(nodes_df, arcs_df, weight = 'trip_time')
plot_taxi_route(G, opt_paths, nodes_df)

## Part 2: Bipartite Graph Formulation (At Scale)

Now, instead of selecting a sample selection of taxi trips, try filter the trips by time window of interest. The following example selects all the trips from 5 pm to 5:15 pm.

In [ ]:
# Filter trips by time window of interest
start_time = 1020
end_time = 1035
trips = trips_df.copy()
trips = trips[(trips.start_time >= start_time) & 
              (trips.start_time + trips.trip_time <= end_time)].copy()
trips.start_time = trips.start_time - start_time

Nodes and edges are defined similarly.

In [ ]:
# Intialize nodes and edges
DO_nodes = list()
PU_nodes = list()
edges = list()
# Initialize a dict that maps a PU node to a DO node
PUtoDO = dict()

In [ ]:
# Specify nodes - each node is a tuple of (location_id, time, trip_id, "DO"/"PU")
for index, row in trips.iterrows():
    s = row['start_node']
    t = row['end_node']
    s_t = row['start_time']
    t_t = s_t + row['trip_time']
    DO_node = (int(t), t_t, index, 'DO')
    PU_node = (int(s), s_t, index, 'PU')
    DO_nodes.append(DO_node)
    PU_nodes.append(PU_node)
    PUtoDO[PU_node] = DO_node

In [ ]:
# Sort the nodes by time
DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
PU_nodes = sorted(PU_nodes, key = lambda x: x[1])

# Specify edges
max_waiting_time = 10

for DO_node in DO_nodes:
    for PU_node in PU_nodes:
        if PU_node[1] > DO_node[1] + max_waiting_time:
            break
        else:
            if PU_node[1] >= DO_node[1]:
                time = times_df.at[(DO_node[0], PU_node[0])]
                if ((PU_node[1] - DO_node[1]) - max_waiting_time  <= time) & (time <= (PU_node[1] - DO_node[1])):
                    edges.append((DO_node, PU_node))

In [ ]:
# load the model
B = nx.Graph()
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(DO_nodes, bipartite=0)
B.add_nodes_from(PU_nodes, bipartite=1)
# Add edges only between nodes of opposite node sets
B.add_edges_from(edges)

In [ ]:
match = nx.bipartite.maximum_matching(B, DO_nodes)

In [ ]:
print('size of max cardinality matching:', len(match) / 2)

In [ ]:
# Calculate number of unmatched nodes (minimum number of taxis needed)
# TODO: Assign num_taxi with the minimum number of taxis needed
# num_taxi = XXX

### BEGIN SOLUTION
num_taxi = len(DO_nodes) - len(match)/2
### END SOLUTION

print('min number of taxis needed: ', num_taxi)

The maximum matching problem can also be formulated via an IP formulation.
$$\begin{align*}
\max \quad & \sum_{e \in E}x_e \\
\text{s.t.} \quad &  \sum_{e \in \delta(v)} x_e \leq 1 \quad \forall v \in P \cup D & (1)\\
\quad & x_e \in \{0,1\} \quad \forall e \in E & (2)\\
\end{align*}$$

where, $\delta(v)$ is the set of edges incident on the vertex $v \in P \cup D$.

In [ ]:
# A dictionary that stores the list of edges adjacent to node
incident = dict()
for v in (DO_nodes + PU_nodes):
    incident[v] = [edge for edge in edges if v in edge]

In [ ]:
solver = OR.Solver('taxi_bipartite', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
# Decision variables
x = {}  
for e in B.edges:
    x[e] = solver.IntVar(0, 1, ('(%s)' % str(e)))

solver.Maximize(sum(x[e] for e in edges))

for v in (DO_nodes + PU_nodes):
    # TODO: Specify constraint (1)

    ### BEGIN SOLUTION
    if v in incident.keys():
        solver.Add(sum(x[e] for e in incident[v]) <= 1)
    ### END SOLUTION

In [ ]:
solver.Solve()
print('Solution:')
print('Objective value =', solver.Objective().Value())

We can also check our answer by reducing the max matching problem to a max flow problem - the max flow value should be equal to the size of max cardinality matching.

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(PU_nodes + DO_nodes + ['s'] + ['t'])
for edge in edges:
    G.add_edge(edge[0], edge[1], capacity = 1)
for DO_node in DO_nodes:
    G.add_edge('s', DO_node, capacity = 1)
for PU_node in PU_nodes:
    G.add_edge(PU_node, 't', capacity = 1)

In [ ]:
flow_value, flow_dict = nx.maximum_flow(G, "s", "t")

In [ ]:
print('max flow value: ', flow_value)

In [ ]:
# Try another alogrithm to solve max flow
from networkx.algorithms.flow import shortest_augmenting_path
print('max flow value: ', nx.maximum_flow(G, "s", "t", flow_func=shortest_augmenting_path)[0])

We can turn the process of building a bipartite graph and solving the max cardinality matching into a function

In [ ]:
def max_match(start_time, end_time, trips_df, max_waiting_time):
    # filter trips by time window of interest
    trips = trips_df.copy()
    trips = trips[(trips.start_time >= start_time) & 
                  (trips.start_time + trips.trip_time <= end_time)].copy()
    trips.start_time = trips.start_time - start_time

    # create a dictionary to map the location pairs to trip time
    loc_time = dict()
    for index, row in arcs_df.iterrows():
        i = row['start']
        j = row['end']
        delay = row['trip_time']
        loc_time[(i, j)] = delay
    # Intialize nodes and edges
    DO_nodes = list()
    PU_nodes = list()
    edges = list()
    # Initialize a dict that maps a PU node to a DO node
    PUtoDO = dict()
    # Specify nodes - each node is a tuple of (location_id, time, trip_id, "DO"/"PU")
    for index, row in trips.iterrows():
        s = row['start_node']
        t = row['end_node']
        s_t = row['start_time']
        t_t = s_t + row['trip_time']
        DO_node = (int(t), t_t, index, 'DO')
        PU_node = (int(s), s_t, index, 'PU')
        DO_nodes.append(DO_node)
        PU_nodes.append(PU_node)
        PUtoDO[PU_node] = DO_node
    DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
    PU_nodes = sorted(PU_nodes, key = lambda x: x[1])
    
    # Specify edges
    for DO_node in DO_nodes:
        for PU_node in PU_nodes:
            if PU_node[1] > DO_node[1] + max_waiting_time:
                break
            else:
                if PU_node[1] >= DO_node[1]:
                    time = times_df.at[(DO_node[0], PU_node[0])]
                    if ((PU_node[1] - DO_node[1]) - max_waiting_time <= time) & (time <= (PU_node[1] - DO_node[1])):
                        edges.append((DO_node, PU_node))
    
    # load the model
    B = nx.Graph()
    # Add nodes with the node attribute "bipartite"
    B.add_nodes_from(DO_nodes, bipartite=0)
    B.add_nodes_from(PU_nodes, bipartite=1)
    # Add edges only between nodes of opposite node sets
    B.add_edges_from(edges)
    
    top_nodes = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0}
    match = nx.bipartite.maximum_matching(B, top_nodes)
    num_taxi = len(DO_nodes) - len(match)/2

    return B, match, num_taxi, trips

In [ ]:
fifteen_min_B, fifteen_min_match, fifteen_min_vnum_taxi, fifteen_min_trips = max_match(1020, 1035, trips_df, 10)
print('max cardinality matching:', len(fifteen_min_match)/2)
print('min number of taxis needed to cover all trips:', fifteen_min_vnum_taxi)

Try expanding the time window to 5-6 pm and restricting the max waiting time to 5 mininutes.

In [ ]:
# TODO: Uncomment the code below and change START_TIME, END_TIME, and MAX_WAIT_TIME.
# one_hr_B, one_hr_match, one_hr_num_taxi, one_hr_trips =  = max_match(START_TIME, END_TIME, trips_df, MAX_WAIT_TIME)

### BEGIN SOLUTION
one_hr_B, one_hr_match, one_hr_num_taxi, one_hr_trips = max_match(1020, 1080, trips_df, 5)
### END SOLUTION
print('max cardinality matching:', len(one_hr_match)/2)
print('min number of taxis needed to cover all trips:', one_hr_num_taxi)

## Part 3: Compare one day's taxi routing solutions

In [ ]:
# Load the result of a day's data with a max waiting time of 10 min
with open('data/day_match.pkl', 'rb') as f:
    match = pickle.load(f)

In [ ]:
# Get the optimal taxi paths
opt_paths = match_to_path(match, trips_df)

In [ ]:
# Construct the street network
G = street_network(nodes_df, arcs_df, weight = 'trip_time')

In [ ]:
# Select all the routes with more than 1 trip
opt_paths2 = []
for path in opt_paths:
    if len(path) > 1:
        opt_paths2.append(path)

In [ ]:
plot_taxi_route(G, opt_paths2[:5], nodes_df,'Optimal Sample Taxi Routes')

Retrieve the original paths of the taxis

In [ ]:
og_paths = get_og_path(trips_df)

In [ ]:
# Select all the routes with more than 1 trip
og_paths2 = []
for path in og_paths:
    if len(path) > 1:
        og_paths2.append(path)

In [ ]:
plot_taxi_route(G, og_paths2[:5], nodes_df,'Original Sample Taxi Routes')

Compare statistics of the taxi routings

In [ ]:
opt_stats = get_taxi_stats(opt_paths, trips_df)
og_stats = get_taxi_stats(og_paths, trips_df)

Summary statistics for the original taxi routing

In [ ]:
agg_stats(og_stats)

Summary statistics for the optimal taxi routing

In [ ]:
agg_stats(opt_stats)

In [ ]:
plot_stats(og_stats, opt_stats)

**Q:** What conclusions can you draw from the histograms above in terms of the difference between the original and the optimal taxi routings?

**A:** <font color='blue'> Total trip time and empty trip time are greatly reduced. However, to offset the reductions in trip time, the number of trips and on-trip percentage are increased.</font>

Another way to compare different taxi routings is to compare their distribution of taxis over time.

In [ ]:
day_data = get_day_dist(og_paths, opt_paths, times_df)

In [ ]:
plot_taxi_dist(day_data)

**Q:** What are the estimated original and optimal number of circulating taxis on this example day shown above?

**A:** <font color='blue'> Original number of circulating taxis: 7798; optimal number of circulating taxis: 1305.  </font>

## Part 4: Minimizing the number of taxis while accounting for the waiting time of the drivers

In the bipartite graphs we formulated, the edges actually contain information of the waiting time. By cleverly incorporating the waiting time information into the weights of the edges, we can include secondary objectives in addition to the primary objective of minimizing the number of taxis. 

In [ ]:
# Locate six example trips
ex_trips = [68634, 68536, 68894, 69699, 69963, 70937]
trips = trips_df.iloc[ex_trips]

In [ ]:
# Find a base max card matching of the six example trips
B, match, num_taxi, trips = max_match(0, 1440, trips, 30)

To achieve fair distribution of taxi trips, we want to minimize the maximum waiting time between any two trips. Minimizing the maximum waiting time is equivalent to maximizing the minimum edge wait (negative of waiting time).
A max-min matching (or a bottleneck matching) problem can be constructed as an IP:
     $$\begin{align*}
    \max_E \quad  \min \quad & \{c_ex_e|e \in E\} \\
    \text{s.t.} \quad &  \sum_{e \in \delta(v)} x_e \leq 1 \quad \forall v \in P \cup D & (1)\\
    \quad & x_e \in \{0,1\} \quad \forall e \in E & (2)\\
    \end{align*}$$

where, $\delta(v)$ is the set of edges incident on the vertex $v \in P \cup D$, and $c_e$ is the edge weight.

Instead of directly solving an IPA, we can come up with a bisection search algorithm below to find this minimized maximum (or a bottleneck) waiting time.

In [ ]:
def bisection_search(base_graph, base_match):
    '''A bisection search algorithm that finds a max card matching with bottleneck waiting time'''
    max_wait_times = []
    for edge in base_graph.edges(data = True):
        waiting_time = edge[1][1] - edge[0][1] - times_df.at[(edge[0][0], edge[1][0])]
        edge[2]['weight'] = -waiting_time
    b = np.max([-base_graph[x][y]['weight'] for x, y in base_match.items()]) # base max waiting time
    base_match_size = len(base_match)
    a = 0
    
    while a <= b :
        lam = (b + a)//2
        new_graph = copy.deepcopy(base_graph)
        remove = [edge for edge in base_graph.edges(data = True) if -edge[2]['weight'] > lam]
        new_graph.remove_edges_from(remove)
        top_nodes = {n for n, d in new_graph.nodes(data=True) if d["bipartite"] == 0}
        new_match = nx.bipartite.maximum_matching(new_graph, top_nodes)
        new_match_size = len(new_match)
        wait_times = [-new_graph[x][y]['weight'] for x, y in new_match.items()]
        # Every value larger than b gives the maximum cardinality matching
        if base_match_size != new_match_size:
            a = lam + 1
        else:
            b = lam - 1
    # max wait time should be b + 1
    remove = [edge for edge in base_graph.edges(data = True) if -edge[2]['weight'] > b + 1]
    bottleneck_graph = copy.deepcopy(base_graph)
    bottleneck_graph.remove_edges_from(remove)
    top_nodes = {n for n, d in bottleneck_graph.nodes(data=True) if d["bipartite"] == 0}
    bottleneck_match = nx.bipartite.maximum_matching(new_graph, top_nodes)
    return bottleneck_match, bottleneck_graph, b + 1

In [ ]:
bottleneck_match, bottleneck_graph, bottleneck_max_wait_time = bisection_search(B, match)
print(f'Bottleneck max wait time is: {bottleneck_max_wait_time}mins')
plot_weight_bipartite(B, bottleneck_match, match_to_path(bottleneck_match, trips), power = 0, with_labels = True, edge_labels = True)

We might be also interested in knowing the max cardinality matching that minimizes the sum of the waiting time, or possibly even the power sum of the waiting time. This can be formulated as a min cost matching problem. 

The IP formulation of the min cost matching problem is shown as the following:
$$\begin{align*}
\min \quad & \sum_{e \in E}c_ex_e \\
\text{s.t.} \quad &  \sum_{e \in \delta(v)} x_e \leq 1 \quad \forall v \in P \cup D & (1)\\
\quad & x_e \in \{0,1\} \quad \forall e \in E & (2)\\
\end{align*}$$

where, $\delta(v)$ is the set of edges incident on the vertex $v \in P \cup D$, and $c_e$ is the edge weight （some power of the waiting time)

In [ ]:
def min_cost_match(B, power):
    '''Return a max card matching with the minimimum sum of the power of the waiting time'''
    # Append weight (negative of the power of waiting time) to each edge
    if power != 0:
        for edge in B.edges(data = True):
            waiting_time = edge[1][1] - edge[0][1] - times_df.at[(edge[0][0], edge[1][0])]
            edge[2]['weight'] = -waiting_time **power
        weight_match_tuple = nx.max_weight_matching(B, maxcardinality=True, weight='weight')
        weight_match = dict()
        for x, y in weight_match_tuple:
            weight_match[x] = y
            weight_match[y] = x
        wait_times = [(-B[x][y]['weight'])**(1/power) for x, y in weight_match.items()]
        total_wait_time = np.sum(wait_times)/2
        max_wait_time = np.max(wait_times)
#         print(f'total waiting time: {total_wait_time} mins')
        print(f'maximum waiting time: {max_wait_time} mins')

    else:
        for edge in B.edges(data = True):
            waiting_time = edge[1][1] - edge[0][1] - times_df.at[(edge[0][0], edge[1][0])]
            edge[2]['weight'] = -waiting_time
        weight_match = nx.bipartite.maximum_matching(B, top_nodes = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0})
        wait_times = [int(-B[x][y]['weight']) for x, y in weight_match.items()]
        total_wait_time = np.sum(wait_times)/2
        max_wait_time = np.max(wait_times)
#         print('total waiting time: {0:.2f} mins'.format(total_wait_time))
        print('maximum waiting time: {0:.2f} mins'.format(max_wait_time))
    return weight_match

In [ ]:
# minimizing the total waiting time
min_sum_match =  min_cost_match(B, 1)
plot_weight_bipartite(B, min_sum_match, match_to_path(min_sum_match, trips), power = 1, with_labels = True, edge_labels = True)

In [ ]:
min_square_sum_match =  min_cost_match(B, 2)
plot_weight_bipartite(B, min_square_sum_match, match_to_path(min_square_sum_match, trips), power = 2, with_labels = True, edge_labels = True)

In [ ]:
min_third_power_sum_match =  min_cost_match(B, 3)
plot_weight_bipartite(B,min_third_power_sum_match, match_to_path(min_third_power_sum_match, trips), power = 3, with_labels = True, edge_labels = True)

**Q:** What do you notice about the maximum waiting time of the min-cost matching as we increase the power? Can you provide an explanation for this?

**A:** <font color='blue'> The maximum waiting time of the min-cost matching converges to the bottleneck maximum waiting time as we increases the power. This is because increasing the power penalizes larger waiting time. </font>